In [2]:
import twitter_creds
import requests
import base64
from pprint import pprint

In [3]:
def get_bearer_header():
    uri_token_endpoint = 'https://api.twitter.com/oauth2/token'
    key_secret = f"{twitter_creds.consumer_key}:{twitter_creds.consumer_key_secret}".encode('ascii')
    b64_encoded_key = base64.b64encode(key_secret)
    b64_encoded_key = b64_encoded_key.decode('ascii')

    auth_headers = {
        'Authorization': 'Basic {}'.format(b64_encoded_key),
        'Content-Type': 'application/x-www-form-urlencoded;charset=UTF-8'
    }

    auth_data = {
        'grant_type': 'client_credentials'
    }

    auth_resp = requests.post(uri_token_endpoint, headers=auth_headers, data=auth_data)
    bearer_token = auth_resp.json()['access_token']

    bearer_header = {
       'Accept-Encoding': 'gzip',
       'Authorization': 'Bearer {}'.format(bearer_token),
       'oauth_consumer_key': twitter_creds.consumer_key 
    }
    return bearer_header

In [4]:
def getConversationId(id):
    uri = 'https://api.twitter.com/2/tweets?'

    params = {
        'ids':id,
        'tweet.fields':'conversation_id',
        'max_results': 100
    }

    bearer_header = get_bearer_header()
    resp = requests.get(uri, headers=bearer_header, params=params)
    return resp.json()['data'][0]['conversation_id']

In [19]:

#   --url 'ids=1225917697675886593&tweet.fields=author_id,conversation_id,created_at,in_reply_to_user_id,referenced_tweets&expansions=author_id,in_reply_to_user_id,referenced_tweets.id&user.fields=name,username' \
#   --header 'Authorization: Bearer $BEARER_TOKEN' 


def getConversation(conversation_id):
    data = {}
    uri = 'https://api.twitter.com/2/tweets/search/recent?'

    params = {
        'query': f'ids:{conversation_id}',
        'tweet.fields': 'author_id,conversation_id,created_at,in_reply_to_user_id,referenced_tweets', 
        'expansions': 'author_id,in_reply_to_user_id,referenced_tweets.id&user.fields=name,username',
        'max_results': 100
    }

    bearer_header = get_bearer_header()
    token = ''
    while token != None:
        if token != '':
            params['next_token'] = token
        resp = requests.get(uri, headers=bearer_header, params=params).json()
        try:
            token = resp['meta']['next_token']
        except:
            break
        if not data:
            print("empty")
            data['data'] = resp['data']
        else:
            print("updating")
            data['data'].extend(resp['data'])
        print(resp['meta']['result_count'])
    return data

In [14]:
getConversationId(1478993643516018692)

'1478993643516018692'

In [20]:
conversations = getConversation(1478993643516018692)

In [21]:
conversations['data'][:10]

KeyError: 'data'